# System Modeling and Dispatching

This notebook is intended to show a power system modeling framework that exploits the capabilities of Julia to improve performance and allow modelers to develop modular system to analyze problems with different complexities. 

The example system for this notebook is the [Small Test Systems for Power System Economic Studies](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=5589973), modified to include 2 PV-Plants and time series of load and renewable energy. The system data can also be accessed in [Matpower format](http://www.pserc.cornell.edu/matpower/docs/ref/matpower5.0/case5.html). 

## 5 - Bus system example

It is possible to store the data for the 5-bus system described in the later sections in terms of the aforementioned type structure. 

The one-line diagram of the system is as follows, where the peak load of each bus is shown: 

![5bus_system](5bus.png)

First the nodes and system parameters are defined in terms of the system and bus types. Which eventually allows for the re-use of the same information is some other components of the system change. 

## Intro
The objecitve is to exploit Julia's integration of dynamic types with the function dispatch. As explained in Julia's documentation: 

"Julia’s type system is dynamic, but gains some of the advantages of static type systems by making it possible to indicate that certain values are of specific types. This can be of great assistance in generating efficient code, but even more significantly, it allows method dispatch on the types of function arguments to be deeply integrated with the language."

The way the types are defined for MEMF is by using immutable types. There are two kinds of composite types (in 0.6) ``struct`` (an immutable type, old name was ``immutable``) and ``mutable struct`` (a mutable type, old name was ``type``). ``mutable struct``s are always allocated on the heap. ``struct``s will be allocated on the stack under certain conditions. (One case where they won't be allocated on the stack right now is if they have a field that is a ``mutable struct``). If a ``struct`` only has e.g. bitstype fields, the ``struct`` should always end up on the stack, and for example an array of such a ``struct`` should have a really nice dense memory layout.

For more details on Julia types, refer to the [documentation](https://docs.julialang.org/en/release-0.6/manual/types/)

## Environment and packages

The examples in this notebook depend upon Julia 1.1 and a specific set of package releases as defined in the `env` folder. Also, the build process of `PowerSimulations.jl` retrieves a few sample datafiles that will be used for demonstration. The following steps outline loading the environment, building `PowerSimulations.jl`, and loading the required packages.

In [1]:
] activate env; instantiate; st

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
    Status `~/Documents/repos/Examples/env/Project.toml`
  [5ae59095] Colors v0.9.5
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.18.0
  [e2685f51] ECOS v0.9.4
  [60bf3e95] GLPK v0.9.1
  [2030c09a] InfrastructureModels v0.2.0 #moi-2 (https://github.com/lanl-ansi/InfrastructureModels.jl.git)
  [b6b21f68] Ipopt v0.5.4
  [4076af6c] JuMP v0.19.0+ #master (https://github.com/JuliaOpt/JuMP.jl.git)
  [51fcb6bd] NamedColors v0.2.0
  [774612a8] ParameterJuMP v0.0.1 #dfb1e3c (https://github.com/JuliaStochOpt/ParameterJuMP.jl.git)
  [f0f68f2c] PlotlyJS v0.12.3
  [91a5bcdd] Plots v0.24.0
  [c36e90e8] PowerModels v0.10.0 #moi-2 (https://github.com/lanl-ansi/PowerModels.jl.git)
  [e690365d] PowerSimulations v0.1.0 #master (https://github.com/nrel/powersimulations.jl)
  [bcd98974] PowerSystems v0.3.0
  [9e3dc215] TimeSeries v0.14.1


In [20]:
# might have to do this the first time

In [21]:
] build PowerSystems

  Building CodecZlib ───→ `~/.julia/packages/CodecZlib/DAjXH/deps/build.log`
  Building PowerSystems → `~/.julia/packages/PowerSystems/6RPfG/deps/build.log`


In [2]:
using PowerSystems;
using PowerSimulations;
using JuMP
using TimeSeries;
using GLPK; 
using DataFrames;
using PlotlyJS;
using D3TypeTrees;

┌ Info: Precompiling PlotlyJS [f0f68f2c-4968-5e81-91da-67840de0976a]
└ @ Base loading.jl:1186


HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-15125283334365475585\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-10206545526933230328\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/4d683eefb9c701cf328044d08f5830616023eb05-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-15125283334365475585\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

## Types in PowerSystems

The following trees are made with [D3TypeTrees](https://github.com/claytonpbarrows/D3TypeTrees.jl), nodes that represent Structs will show the Fields in the hoverover tooltip.

In [5]:
fieldnames(System)

(:buses, :generators, :loads, :branches, :storage, :basepower, :time_periods)

In [7]:
supertype(PowerSystems.Component)

PowerSystemType

In [8]:
TypeTree(PowerSystems.Component, init_expand=10)

Component
├──Service
│  ├──Reserve
│  │  ├──StaticReserve (0 children)
│  │  └──ProportionalReserve (0 children)
│  └──Transfer
├──PowerSystems.TechnicalParams
│  ├──TechRenewable
│  ├──EconHydro
│  ├──TechThermal
│  ├──TechHydro
│  ├──EconRenewable
│  └──EconThermal
├──GenClasses
├──Forecast
│  ├──Scenarios
│  ├──Probabilistic
│  └──Deterministic
└──Device
   ├──Branch
   │  ├──TapTransformer (0 children)
   │  ├──HVDCLine (0 children)
   │  ├──DCLine (1 children)
   │  ├──PhaseShiftingTransformer (0 children)
   │  ├──Transformer2W (0 children)
   │  ├──Line (0 children)
   │  └──MonitoredLine (0 children)
   └──Injection
      ├──Storage (1 children)
      ├──LoadZones (0 children)
      ├──Bus (0 children)
      ├──Generator (3 children)
      └──ElectricLoad (3 children)

In [9]:
TypeTree(PowerSystems.Service)

Service
├──Reserve
│  ├──StaticReserve
│  └──ProportionalReserve
└──Transfer

In [10]:
TypeTree(Forecast)

Forecast
├──Scenarios
├──Probabilistic
└──Deterministic

## Load some data

In [11]:
base_dir = joinpath(dirname(dirname(pathof(PowerSystems))),"data")
include(string(base_dir,"/data_5bus.jl"));

In [13]:
sys = System(nodes5, generators5, loads5_DA, branches5,nothing,230.0)

┌ Warning: Data for line rating is 20 times larger than the base MVA for the system
│ . Power Systems inferred the Data Provided is in MVA and will transform it using a base of basemva
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/utils/IO/branchdata_checks.jl:90
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/packages/PowerSystems/Biewz/src/utils/IO/branchdata_checks.jl:19
┌ Warning: The angle limits provided is larger than 3π/2 radians.
│  PowerSystems inferred the data provided in degrees and will transform it to radians
└ @ PowerSystems /Users/cbarrows/.julia/p

System:
   buses: Bus[Bus(name="nodeA"), Bus(name="nodeB"), Bus(name="nodeC"), Bus(name="nodeD"), Bus(name="nodeE")]
   generators: 
     GenClasses(T:5,R:2,H:0):
   thermal: ThermalDispatch[ThermalDispatch(name="Alta"), ThermalDispatch(name="Park City"), ThermalDispatch(name="Solitude"), ThermalDispatch(name="Sundance"), ThermalDispatch(name="Brighton")]
   renewable: RenewableGen[RenewableFix(name="SolarBusC"), RenewableCurtailment(name="WindBusA")]
   hydro: nothing
     (end generators)
   loads: ElectricLoad[PowerLoad(name="Bus2"), PowerLoad(name="Bus3"), PowerLoad(name="Bus4"), InterruptibleLoad(name="IloadBus4")]
   branches: Line[Line(name="1"), Line(name="2"), Line(name="3"), Line(name="4"), Line(name="5"), Line(name="6")]
   storage: nothing
   basepower: 230.0
   time_periods: 24

## Information about the nodes

The node structure include both topological information and device information. 

```Julia 
using PowerSystems
Node_direct = Bus(1,"nodeA", "PV", 0, 1.0, @NT(min = 0.90, max = 1.1), 230)
```
The node data can be added directly or using named fields. This allows the users to have more clarity when inputing the data. Using named fields, it is possible to only define the minimum fields necessary for the desired analysis. 

For example, in a dispatch problem with not network model, a bus can be specified as follows:

```Julia
Node_dispatch = Bus(number = 1, name = "City")
```

However, if analysis includes an AC Power Flow calculation, then the bus can be specified in much more detail:

```Julia 
Node_acpf = Bus(number = 1, 
                name = "City", 
                bustype = "PV", 
                angle = 0.3, 
                voltage = 0.95, 
                voltagelims = @NT(min = 0.90, max = 1.1))
```

In this more detailed representation of the node, voltage limits are included (```@NT``` stands for Named Tuple). This notation will dissapear with Julia V0.7

The full AC data for the 5-bus system is as follows:

In [14]:
nodes5

5-element Array{Bus,1}:
 Bus(name="nodeA")
 Bus(name="nodeB")
 Bus(name="nodeC")
 Bus(name="nodeD")
 Bus(name="nodeE")

# Information about the Branches and Network

The implementation of the branch data and network explots the use of Julia's abstract types. Both transformers and lines are part of the abstract type Branch. The hierarchy is as follows:

In [15]:
TypeTree(Branch)

Branch
├──TapTransformer
├──HVDCLine
├──DCLine
│  └──VSCDCLine
├──PhaseShiftingTransformer
├──Transformer2W
├──Line
└──MonitoredLine

Any branch is described in terms of its connection points and physical parameters. Where the field ```connectionpoints``` only accepts elements of the type ```Bus``` enforcing consistency between the information. Another feature of this system is the use of abstract types to create collections of type that share the same data structures. 

This representation allows the inclusion of topological information and line characteristics in the same strucutre. For example

```Julia 
Line = Line("1", true, (nodes5[1],nodes5[2]), 0.00281, 0.0281, 0.00712, 400.0, nothing)
```

The implementation of the line structure also uses named fields. It also allows creating lines with less fields. 

```Julia 
Line = Line(name = "LA - San Diego", 
            status = true, 
            (node1, node2), 
            R=0.02, 
            X=0.2, 
            b=0,
            rate = nothing, 
            anglelimits = @NT(min = -10, max = 10)
```

The full AC data for a line between buses A and B with no angle limits is as follows:

In [18]:
branches5[1]

Line:
   name: 1
   available: true
   connectionpoints: (from = Bus(name="nodeA"), to = Bus(name="nodeB"))
   r: 0.00281
   x: 0.0281
   b: (from = 0.00356, to = 0.00356)
   rate: 38.038742043967325
   anglelimits: (min = -0.7853981633974483, max = 0.7853981633974483)

The whole system network characteristics are summarizes in the network structure. ```PowerSystems.jl``` uses an inner constructor to calculate the fields relevant for power systems analisys. 

The Network structure has a field for Ybus, Incidence Matrix and PTDF matrix. 

```Julia 
Net5 = Network(FiveBus, branches5, nodes5); 
```


The incidence matrix and PTDF are as follows: 

In [20]:
PTDF, A = PowerSystems.buildptdf(branches5,nodes5);
@show PTDF;
@show A;

PTDF = [0.193917 -0.475895 -0.348989 0.0 0.159538; 0.437588 0.258343 0.189451 0.0 0.36001; 0.368495 0.217552 0.159538 0.0 -0.519548; 0.193917 0.524105 -0.348989 0.0 0.159538; 0.193917 0.524105 0.651011 0.0 0.159538; -0.368495 -0.217552 -0.159538 0.0 -0.480452]
A = [1.0 1.0 1.0 0.0 0.0 0.0; -1.0 0.0 0.0 1.0 0.0 0.0; 0.0 0.0 0.0 -1.0 1.0 0.0; 0.0 -1.0 0.0 0.0 -1.0 1.0; 0.0 0.0 -1.0 0.0 0.0 -1.0]


In [21]:
PowerSystems.buildlodf(branches5,nodes5)

6×6 Array{Float64,2}:
 -1.0        0.344795   0.307071  -1.0       -1.0       -0.307071
  0.542857  -1.0        0.692929   0.542857   0.542857  -0.692929
  0.457143   0.655205  -1.0        0.457143   0.457143   1.0     
 -1.0        0.344795   0.307071  -1.0       -1.0       -0.307071
 -1.0        0.344795   0.307071  -1.0       -1.0       -0.307071
 -0.457143  -0.655205   1.0       -0.457143  -0.457143  -1.0     

# Load Data 

A similar type definition is used for renewable power, storage systems and loads. The main difference in the structures of load and renewable power is the inclusion of a field for time series information of the type ```TimeArray```. This is included considering that in modern power systems analysis for the integration energy sources accounting for the time component of load and generation is paramount.  

Also, type hierarchy can be exploited to represent a larger family of load types:

In [22]:
TypeTree(ElectricLoad)

ElectricLoad
├──ControllableLoad
│  └──InterruptibleLoad
├──FixedAdmittance
└──StaticLoad
   └──PowerLoad

A classic static load is as follows:

```Julia 

Load = StaticLoad("Bus2", true, nodes5[2], "P", 300, 98.61, TimeArray(DayAhead, loadbus2_ts_DA))

```

But it can also be defined with named fields as follows: 

```Julia 
Load = StaticLoad(name = "Denver", 
                  status = true, 
                  bus = nodes5[2], 
                  model = "P", 
                  maxrealpower = 400,  
                  maxreactivepower = 30, 
                  scalingfactor = TimeArray(DayAhead, loadbus2_ts_DA)
```

In the 5-bus cases shown in this notebook, all the loads have been implemented as ```StaticLoads```, further in the notebook the implementation of an interruptible load is shown.

In [23]:
loads5_DA[1]

PowerLoad:
   name: Bus2
   available: true
   bus: Bus(name="nodeB")
   maxactivepower: 300.0
   maxreactivepower: 98.61
   scalingfactor: 24×1 TimeArray{Float64,1,DateTime,Array{Float64,1}} 2024-01-01T00:00:00 to 2024-01-01T23:00:00

Using ```TimeArrays``` in Julia enable a whole set of analytical tools relevant to Renewable Energy Integration. It is possible to calculate the percent changes in the time series with the embedded functions. For instance the percent change over the time series.

In [24]:
percentchange(loads5_DA[1].scalingfactor)

23×1 TimeArray{Float64,1,DateTime,Array{Float64,1}} 2024-01-01T01:00:00 to 2024-01-01T23:00:00
│                     │ A       │
├─────────────────────┼─────────┤
│ 2024-01-01T01:00:00 │ -0.0877 │
│ 2024-01-01T02:00:00 │ -0.0169 │
│ 2024-01-01T03:00:00 │ -0.0468 │
│ 2024-01-01T04:00:00 │ -0.0139 │
│ 2024-01-01T05:00:00 │ 0.0051  │
│ 2024-01-01T06:00:00 │ 0.0237  │
│ 2024-01-01T07:00:00 │ 0.0352  │
│ 2024-01-01T08:00:00 │ 0.0625  │
│ 2024-01-01T09:00:00 │ 0.0556  │
│ 2024-01-01T10:00:00 │ 0.0369  │
│ 2024-01-01T11:00:00 │ 0.0151  │
│ 2024-01-01T12:00:00 │ 0.0056  │
│ 2024-01-01T13:00:00 │ -0.0124 │
│ 2024-01-01T14:00:00 │ -0.014  │
│ 2024-01-01T15:00:00 │ -0.0073 │
│ 2024-01-01T16:00:00 │ 0.0087  │
│ 2024-01-01T17:00:00 │ 0.0991  │
│ 2024-01-01T18:00:00 │ 0.093   │
│ 2024-01-01T19:00:00 │ 0.0101  │
│ 2024-01-01T20:00:00 │ -0.0088 │
│ 2024-01-01T21:00:00 │ -0.0307 │
│ 2024-01-01T22:00:00 │ -0.041  │
│ 2024-01-01T23:00:00 │ -0.0917 │

Time Series also can be easily plotted (In the near future we expect to have a better way to plot Time Series and Power Systems) 

In [25]:
plot(loads5_DA[1].maxactivepower*values(loads5_DA[1].scalingfactor))

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with field margin"

## Generation Data 

Generators are organized in the data model the three categories:

* **Thermal:** Can represen Coal, Gas, CCGT, Biomass. 
* **Hydro:** Two subtypes Dispatchable, Non Dispatchable. 
* **Renewable:** Fixed Output, Curtailable, Reactive Power Dispatch.  


In [26]:
TypeTree(PowerSystems.Generator)

Generator
├──RenewableGen
│  ├──RenewableFix
│  ├──RenewableCurtailment
│  └──RenewableFullDispatch
├──HydroGen
│  ├──HydroFix
│  ├──HydroCurtailment
│  └──HydroStorage
└──ThermalGen
   ├──ThermalDispatch
   └──ThermalGenSeason

Thermal generators are characterized depending on the type of analysis that is being done. The data is split in technical and economical parameters as follows: 

In [27]:
generators5[2]

ThermalDispatch:
   name: Park City
   available: true
   bus: Bus(name="nodeA")
   tech: TechThermal
   econ: EconThermal

In order to make the implementation of the Data Model easier, generators can also be created using named fields as follows: 

```Julia 

TechThermal(; realpower = 80, 
          realpowerlimits = (max = 100, min = 45), 
          reactivepower = nothing,  
          reactivepowerlimits = nothing,
          ramplimits = nothing,
          timelimits = nothing
        ); 

EconThermal(;   capacity = 100, 
            variablecost = x -> 0.04303*x^2 + 20*x,
            fixedcost = 0.0,
            startupcost = 1.0,
            shutdncost = 0.0,
            annualcapacityfactor = nothing
        )



```

One of the interesting features of `PowerSystems.jl` is the flexibility in the representation of the variable cost. In the 5Bus system, variable cost is represented as a single value assumimg that the cost is a linear function. However, as shown in the code above, the cost function can also be represented explictly as a function. This generates more possibilities for analyzing the system data. For example: 

In [28]:
TestGen = ThermalDispatch("Bus1", true, nodes5[1],
                TechThermal(200, (min=0.0, max=200.0), -16.9, (min=-990.0, max=990.0), nothing, nothing),
                EconThermal(40, x -> 0.04303*x^2 + 20*x, 0.0, 0.0, 0.0, nothing)
                )

ThermalDispatch:
   name: Bus1
   available: true
   bus: Bus(name="nodeA")
   tech: TechThermal
   econ: EconThermal

### Renewable Generation Data

Renewable generation is defined with the same principle as loads, time series is a fundamental component for modern energy integration analysis. In the same fashion as with the branches, the information about the bus is included in the definition of the generator, revealing the explicit topological relashionship of the system

In [29]:
generators5[6]

RenewableFix:
   name: SolarBusC
   available: true
   bus: Bus(name="nodeC")
   tech: TechRenewable
   scalingfactor: 24×1 TimeArray{Float64,1,DateTime,Array{Float64,1}} 2024-01-01T00:00:00 to 2024-01-01T23:00:00

In [30]:
plot(values(generators5[6].scalingfactor)*generators5[6].tech.installedcapacity)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with field margin"

## Modeling example 

It is possible to use the type structure/schema to build an optimization model. The system parameters are stored in a `SystemParam` type. These parameters include fields that are calculated often such as the number of buses, base voltage, and number of time_periods in the simulation. 

In [31]:
EconomicDispatch = Model()

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.

In [32]:
g_set = [generators5[i].name for i in 1:7]

7-element Array{String,1}:
 "Alta"     
 "Park City"
 "Solitude" 
 "Sundance" 
 "Brighton" 
 "SolarBusC"
 "WindBusA" 

In [33]:
@variable(EconomicDispatch, Pg[g_set, t = 1:sys.time_periods] >=0 )

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["Alta", "Park City", "Solitude", "Sundance", "Brighton", "SolarBusC", "WindBusA"]
    Dimension 2, 1:24
And data, a 7×24 Array{VariableRef,2}:
 Pg[Alta,1]       Pg[Alta,2]       …  Pg[Alta,23]       Pg[Alta,24]     
 Pg[Park City,1]  Pg[Park City,2]     Pg[Park City,23]  Pg[Park City,24]
 Pg[Solitude,1]   Pg[Solitude,2]      Pg[Solitude,23]   Pg[Solitude,24] 
 Pg[Sundance,1]   Pg[Sundance,2]      Pg[Sundance,23]   Pg[Sundance,24] 
 Pg[Brighton,1]   Pg[Brighton,2]      Pg[Brighton,23]   Pg[Brighton,24] 
 Pg[SolarBusC,1]  Pg[SolarBusC,2]  …  Pg[SolarBusC,23]  Pg[SolarBusC,24]
 Pg[WindBusA,1]   Pg[WindBusA,2]      Pg[WindBusA,23]   Pg[WindBusA,24] 

Constraints can be generated individually for the generators indexing using the names and time steps individually

Constraints can be defined in a vectorized way for all the time-steps, however this method has a significant performance impact. For illustrative purposes of using the type structure/schema the upper limit constraints for generator Alta in an Economic Dispatch Model. 

The constraints array can be filled using for loops and it results in a much more efficiente way of creating the model contraints. In this case, since the data for generation contains different generator types besides the conventional thermal

In [34]:
for i = 1:5 # this only add constraints for the thermal generators. Making more complicated the differentiation with RE
    for t = 1:sys.time_periods
        @constraint(EconomicDispatch, Pg[generators5[i].name,t] <= generators5[i].tech.activepowerlimits.max)
    end
end

Now it is possible to visualize the matrix of constraints populated for the thermal generation. 

The most effective way to make the addition of constraints and take advantage of parametric dispatch is to define functions with the constraint model relevant for different generators. This is the method to build large scale optimization models in JuMP. 

In [35]:
EconomicDispatch = Model()
@variable(EconomicDispatch, Pg[g_set, t = 1:sys.time_periods] >=0 )

2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:
    Dimension 1, ["Alta", "Park City", "Solitude", "Sundance", "Brighton", "SolarBusC", "WindBusA"]
    Dimension 2, 1:24
And data, a 7×24 Array{VariableRef,2}:
 Pg[Alta,1]       Pg[Alta,2]       …  Pg[Alta,23]       Pg[Alta,24]     
 Pg[Park City,1]  Pg[Park City,2]     Pg[Park City,23]  Pg[Park City,24]
 Pg[Solitude,1]   Pg[Solitude,2]      Pg[Solitude,23]   Pg[Solitude,24] 
 Pg[Sundance,1]   Pg[Sundance,2]      Pg[Sundance,23]   Pg[Sundance,24] 
 Pg[Brighton,1]   Pg[Brighton,2]      Pg[Brighton,23]   Pg[Brighton,24] 
 Pg[SolarBusC,1]  Pg[SolarBusC,2]  …  Pg[SolarBusC,23]  Pg[SolarBusC,24]
 Pg[WindBusA,1]   Pg[WindBusA,2]      Pg[WindBusA,23]   Pg[WindBusA,24] 

In [37]:
function powerconstraints(m, P_g, Generator::ThermalGen)
    for var in P_g
        @constraint(m, var >= Generator.tech.activepowerlimits.min)
        @constraint(m, var <= Generator.tech.activepowerlimits.max)
    end
end

function powerconstraints(m, P_g, Generator::RenewableGen)
    for (time, var) in enumerate(P_g)
        @constraint(m, var <= Generator.tech.installedcapacity*values(Generator.scalingfactor)[time])
    end
end

powerconstraints (generic function with 2 methods)

In [38]:
for (ix, name) in enumerate(Pg.axes[1])
    if name == generators5[ix].name
        powerconstraints(EconomicDispatch, Pg[name,:], generators5[ix])
    
    else
        error("Bus name in Array and variable do not match")
    end
end

In [39]:
# For now, the only object that is in the object dict is the Pg. the Pmax and Pmin objects are in the model, but not in the ob_dict. PowerSimulations will make this more straghtforward
EconomicDispatch.obj_dict

Dict{Symbol,Any} with 1 entry:
  :Pg => 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:…

In [40]:
c = []
#the first four generators have a varabile cost defined as a julia function
for g in 1:4
    push!(c,generators5[g].econ.variablecost(Pg[g_set[g]]))
end
# The fifth generator has a pwl variable cost, so simplifying to a scalar function
push!(c, Pg[g_set[5]]*generators5[5].econ.variablecost[end][2]/generators5[5].econ.variablecost[end][1])
@objective(EconomicDispatch,Min,sum(c))

14 Pg[Alta,1] + 15 Pg[Park City,1] + 30 Pg[Solitude,1] + 40 Pg[Sundance,1] + 60 Pg[Brighton,1]

In [41]:
EconomicDispatch

A JuMP Model
Minimization problem with:
Variables: 168
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 168 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 120 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 168 constraints
Model mode: AUTOMATIC
CachingOptimizer state: NO_OPTIMIZER
Solver name: No optimizer attached.
Names registered in the model: Pg

In [42]:
optimize!(EconomicDispatch,with_optimizer(GLPK.Optimizer))

In [43]:
EconomicDispatch.moi_backend.state

ATTACHED_OPTIMIZER::CachingOptimizerState = 2

In [44]:
JuMP.primal_status(EconomicDispatch)

FEASIBLE_POINT::ResultStatusCode = 1

In [45]:
JuMP.value(Pg["Alta",1])

0.0

In [46]:
EconomicDispatch.obj_dict

Dict{Symbol,Any} with 1 entry:
  :Pg => 2-dimensional DenseAxisArray{VariableRef,2,...} with index sets:…

# PowerSimulations.jl makes this easier

In [47]:
TypeTree(PowerSimulations.AbstractDeviceFormulation,scopesep="\n")

PowerSimulations
AbstractDeviceFormulation
├──PowerSimulations
│  AbstractThermalFormulation
│  ├──PowerSimulations
│  │  ThermalUnitCommitment
│  └──PowerSimulations
│     AbstractThermalDispatchForm
│     ├──PowerSimulations
│     │  ThermalDispatchNoMin (0 children)
│     ├──PowerSimulations
│     │  ThermalRampLimited (0 children)
│     └──PowerSimulations
│        ThermalDispatch (0 children)
├──PowerSimulations
│  AbstractBranchFormulation
│  ├──PowerSimulations
│  │  AbstractTransformerForm
│  ├──PowerSimulations
│  │  AbstractDCLineForm
│  │  └──PowerSimulations
│  │     SimpleHVDC (0 children)
│  └──PowerSimulations
│     AbstractLineForm
│     ├──PowerSimulations
│     │  PiLine (0 children)
│     └──PowerSimulations
│        SeriesLine (0 children)
├──PowerSimulations
│  AbstractLoadFormulation
│  ├──PowerSimulations
│  │  AbstractControllablePowerLoadForm
│  │  └──PowerSimulations
│  │     InterruptiblePowerLoad (0 children)
│  └──PowerSimulations
│     StaticPowerLoad
├──PowerSimulations
│  AbstractStorageForm
│  └──PowerSimulations
│     BookKeepingModel
├──PowerSimulations
│  AbstractRenewableFormulation
│  ├──PowerSimulations
│  │  AbstractRenewableDispatchForm
│  │  ├──PowerSimulations
│  │  │  RenewableFullDispatch (0 children)
│  │  └──PowerSimulations
│  │     RenewableConstantPowerFactor (0 children)
│  └──PowerSimulations
│     RenewableFixed
└──PowerSimulations
   AbstractHydroFormulation
   ├──PowerSimulations
   │  AbstractHydroDispatchForm
   │  ├──PowerSimulations
   │  │  HydroDispatchRunOfRiver (0 children)
   │  └──PowerSimulations
   │     HydroDispatchSeasonalFlow (0 children)
   ├──PowerSimulations
   │  HydroCommitmentSeasonalFlow
   └──PowerSimulations
      HydroCommitmentRunOfRiver

In [48]:
TypeTree(PowerSimulations.AbstractServiceForm)

UndefVarError: UndefVarError: AbstractServiceForm not defined

In [49]:
TypeTree(PowerSimulations.PowerModels.AbstractPowerFormulation,scopesep="\n")

PowerModels
AbstractPowerFormulation
├──PowerModels
│  AbstractBFForm
│  ├──PowerModels
│  │  AbstractBFQPForm
│  │  └──PowerModels
│  │     SOCBFForm (0 children)
│  └──PowerModels
│     AbstractBFConicForm
│     └──PowerModels
│        SOCBFConicForm (0 children)
├──PowerModels
│  AbstractWRForm
│  ├──PowerModels
│  │  QCWRForm
│  │  └──PowerModels
│  │     QCWRTriForm (0 children)
│  └──PowerModels
│     SOCWRForm
├──PowerModels
│  AbstractACPForm
│  └──PowerModels
│     StandardACPForm
├──PowerModels
│  AbstractLPACForm
│  └──PowerModels
│     AbstractLPACCForm
├──PowerModels
│  AbstractACTForm
│  └──PowerModels
│     StandardACTForm
├──PowerModels
│  AbstractACRForm
│  └──PowerModels
│     StandardACRForm
├──PowerModels
│  AbstractConicPowerFormulation
│  ├──PowerModels
│  │  AbstractWRMForm
│  │  └──PowerModels
│  │     SDPWRMForm (1 children)
│  └──PowerModels
│     AbstractWRConicForm
│     └──PowerModels
│        SOCWRConicForm (0 children)
└──PowerModels
   AbstractActivePowerFormulation
   ├──CopperPlatePowerModel
   └──PowerModels
      AbstractDCPForm
      ├──PowerModels
      │  AbstractDCPLLForm (1 children)
      └──PowerModels
         DCPlosslessForm (2 children)

In [50]:
supertype(PowerSimulations.PowerModels.AbstractPowerFormulation)

Any